## Hämta kyrkogårdar för att koppla samma som

* [GITHUB](https://github.com/salgo60/Gravstensinventeringen-Wikidata)
* WIkidata [Property:P5259](https://www.wikidata.org/wiki/Property:P5259)


In [1]:
from datetime import datetime
start_time  = datetime.now()
print("Last run: ", start_time)

Last run:  2022-11-19 21:18:34.731683


In [2]:
class Gravplats: 
    def __init__(self, namn, url, gravid,lat, lon, kommun, lan, Wikidata):
        self.namn = namn
        self.url = url
        self.gravid = gravid
        self.lat = lat
        self.lon = lon
        self.kommun = kommun
        self.lan = lan
        self.wd = Wikidata
        
    def __repr__(self):
        return f"Grav: ({self.namn!r}, {self.gravid!r}, {self.url!r})"

    def __iter__(self):
        return iter([self.namn, self.url, self.gravid, self.lat,
                    self.lon, self.kommun, self.lan, self.wd])


In [3]:
import json
def getWD (id):
    
    # https://hub.toolforge.org/P5259:begravningsplatser/512?format=json
    baseurl = "https://hub.toolforge.org/P5259:begravningsplatser/"
    huburl = baseurl + str(id) + "?format=json"
    responseHub = requests.get(huburl)
    hubJson = json.loads(responseHub.text)
    
    wd = ""
    if "origin" in hubJson:
        wd = hubJson["origin"]["qid"]
    #print(wd)
    return wd

In [4]:
import requests
import urllib.request
import time,re
from tqdm import tqdm
from bs4 import BeautifulSoup
def cleanupstring(s):
    out_s = s
    while '  ' in out_s:
        out_s = out_s.strip().replace('  ', ' ')
    return out_s, len(s)-len(out_s)

urlbase = "https://www.rotter.se/gsi/begravningsplatser/"

target_string = "?.*"

newList = []
errorList = []
for urlnr in tqdm(range(1,3700)):
    url = urlbase + str(urlnr)
    response = requests.get(url)
    try:
        soup = BeautifulSoup(response.text, "html.parser")
        g_namn = soup.findAll('h1')[0].text
        scriptLat = soup.find_all('script')[27]

        # position: { lat: 58.02754100, lng: 14.66873800 },
        pattern = re.compile(r"lat : ...", re.MULTILINE | re.DOTALL)
        patternLat = re.compile(r"lat(.*)}", re.DOTALL)
        patternKommun = re.compile(r"(.*),kommun", re.DOTALL)

        scriptParse = re.split(patternLat, str(scriptLat))
        glat = scriptParse[1][1:13]
        glon =  scriptParse[1][20:31]

        stringKommun = soup.find_all('h1')[0].find_all_next(text=True)[4]


        kommun = stringKommun.split(",")[0]
        lan = stringKommun.split(",")[1]
        wd = getWD(urlnr)
        gravplats = Gravplats(g_namn,url,urlnr,glat,glon,kommun,lan,wd)
        newList.append(gravplats)
    except :
        #print("error " + url)
        errorList.append(url)
    #time.sleep(1)
    
print (len(newList) ," antal poster")

100%|██████████| 3699/3699 [1:14:45<00:00,  1.21s/it]

3464  antal poster


In [5]:
#newList

In [6]:
import pandas as pd
df = pd.DataFrame(newList)
df.to_csv("GravstensinvKyrkog2.csv")

In [7]:
#df.tail

In [8]:
dfE = pd.DataFrame(errorList)
dfE.to_csv("GravstensinvKyrkog_error.csv")

In [9]:
#newList

In [10]:
end = datetime.now()
print("Ended: ", end)
print('Time elapsed (hh:mm:ss.ms) {}'.format(datetime.now() - start_time))


Ended:  2022-11-19 22:33:20.925618
Time elapsed (hh:mm:ss.ms) 1:14:46.194367
